> Dana jest tablica liczb rzeczywistych wielkości $ n $, reprezentująca kopiec minimum (array-based heap). Mając daną liczbę rzeczywistą $ x $, sprawdź, czy $ k. $ najmniejszy element jest większy lub równy $ x $.

# I Sposób
### (gorszy)

### Omówienie algorytmu

Ten algorytm polega na skorzystaniu z metody, usuwającej najmniejszą wartość z kopca. Nie wymaga on w żaden sposób przeszukiwania kopca lub wymyślania skomplikowanych algorytmów. Konieczne jest jedynie usunięcie $ k-1 $ minimalnych elementów i zwrócenie wartości $ k. $ elementu (tego, który pozostanie w korzeniu drzewa - pod zerowym indeksem tablicy).

Ponieważ nie jest powiedziane, że tablica nie może zostać zmodyfikowana, możemy usuwać z niej elementy. Ja jednak wybrałem implementację kopca, która przerzuca elementy na koniec tablicy, tak, aby ich nie stracić. W poniższej implementacji ograniczam się jedynie do metod klasy, które są konieczne do wykonania tego zadania.

Przyjmuję, że nie istnieje zerowy element, a pierwszy najmniejszy element odpowiada początkowej wartości korzenia.

### Implementacja algorytmu

In [1]:
def is_kth_val_gt_or_eq_x(arr: list, k: int, x):
    # Return None if got not valid value of k
    if k < 1: return None
    heap = MinHeap(arr)
    for _ in range(k-1):
        heap.remove_min()
    return heap.get_min() >= x
        

class MinHeap:
    def __init__(self, values):
        self.heap = values
        self.size = len(values)
        self._free_idx = self.size
        self.build_heap()
    
    def __bool__(self):
        return bool(self._free_idx)
    
    def __len__(self):
        return len(self.heap)
    
    @property
    def heap_size(self):
        return self._free_idx
    
    @staticmethod
    def parent_idx(curr_idx):
        return (curr_idx - 1) // 2
    
    @staticmethod
    def left_child_idx(curr_idx):
        return curr_idx * 2 + 1
    
    @staticmethod
    def right_child_idx(curr_idx):
        return curr_idx * 2 + 2
        
    def get_min(self) -> object:
        return None if not self.heap else self.heap[0]
        
    def remove_min(self) -> object:
        if self.heap_size == 0:
            raise IndexError(f'remove_min from an empty {self.__class__.__name__}')
        # Store a value to be returned
        removed = self.heap[0]
        # Place the last leaf in the root position
        self._free_idx -= 1
        last = self.heap[self._free_idx]
        
        if self.heap_size > 0:
            # Move the last element to the end of an array
            self.heap[self._free_idx] = removed
            self.heap[0] = last
            # Fix a heap in order to stisfy a min-heap property
            self._heapify_down(0, self.heap_size)
        return removed
    
    def swap(self, i, j):
        self.heap[i], self.heap[j] = self.heap[j], self.heap[i]
        
    def _heapify_down(self, curr_idx, end_idx):  # O(log n)
        # Loop till the current node has a child smaller than itself
        # We assume that when we enter a node which both children are
        # larger than this node, a subtree which a current node is a
        # root of must fulfill a min-heap property
        while True:
            l = self.left_child_idx(curr_idx)
            r = self.right_child_idx(curr_idx)
            smallest_idx = curr_idx

            if l < end_idx:
                if self.heap[l] < self.heap[curr_idx]: 
                    smallest_idx = l
                if r < end_idx and self.heap[r] < self.heap[smallest_idx]:
                    smallest_idx = r

            if smallest_idx != curr_idx:
                self.swap(curr_idx, smallest_idx)
                curr_idx = smallest_idx
            else:
                break
                
    def build_heap(self):   # O (n)
        for i in range(self.heap_size // 2 - 1, -1, -1):
            self._heapify_down(i, self.heap_size)

###### Kilka testów

In [2]:
import random

arr = [random.randint(-100, 100) for _ in range(random.randint(1, 20))]
k = random.randrange(1, len(arr) + 1)
x = random.randint(-100, 100)

sorted_arr = sorted(arr)
print("Sorted arr:", sorted_arr, sep='\n')
print('x value:', x)
print("Expected:", sorted_arr[k-1] >= x, f'\t(kth value: {sorted_arr[k-1]})')
print("Result:  ", is_kth_val_gt_or_eq_x(arr, k, x), f'\t(kth value: {arr[0]})')
print("Arr after calling function:", arr, sep='\n')
print("Has an array the same elements as before?:", not set(arr).symmetric_difference(sorted_arr))

Sorted arr:
[-97, -96, -94, -93, -82, -80, -75, -51, -49, -19, -15, -2, 28, 43, 50, 55]
x value: -72
Expected: False 	(kth value: -80)
Result:   False 	(kth value: -80)
Arr after calling function:
[-80, -75, -49, -19, -51, -2, 43, 50, 55, -15, 28, -82, -93, -94, -96, -97]
Has an array the same elements as before?: True


In [3]:
for _ in range(100_000):
    arr = [random.randint(-100, 100) for _ in range(random.randint(1, 20))]
    k = random.randrange(1, len(arr) + 1)
    x = random.randint(-100, 100)
    
    sorted_arr = sorted(arr)
    
    if (sorted_arr[k-1] >= x) is not is_kth_val_gt_or_eq_x(arr, k, x):
        print('Failed for:')
        print('k:', k)
        print('x:', x)
        print(arr)
        break
else:
    print("OK!")

OK!


# II Sposób
### (najlepszy)

### Omówienie algorytmu

##### Wstępne obserwacje

Korzystamy z wiedzy o tym, w jaki sposób są przechowywane wartości w kopcu. Wystarczy zauważyć, że każdy rodzic w kopcu Min Heap ma oboje dzieci o wartości większej od siebie (lub równej sobie). Wystarczy zatem przeszukiwać kopiec, wybierając zawsze to poddrzewo, którego korzeń ma wartość najmniejszą. Nie jest to jednak takie proste, ponieważ może się skończyć dana gałąź i znajdziemy się w liściu, zanim zdążymy odwiedzić $ k-1 $ najmniejszych węzłów. Jest jeszcze jedna kwestia, jak stwierdzić, na której z kolei najmniejszej wartości się znajdujemy obecnie. Nie mamy przecież pewności, że wszystkie węzły, które odrzuciliśmy, wybierając np. lewe poddrzewo, są większe od tych z lewego poddrzewa. Chodzi o to, że wybranie obecnie najmniejszej wartości spośród dzieci bieżącego węzła nie daje nam gwarancji, że wśród dzieci tego węzła również będzie najmniejsza spośród pozostałych wartości. W takiej sytuacji kolejna najmniejsza spośród pozostałych może się znajdować w zupełnie innej części drzewa, reprezentującego kopiec.

##### Plan działania

W tym miejscu warto zauważyć, że najłatwiejszym rozwiązaniem jest rozwiązanie rekurencyjne. Zamiast próbować kolejno odwiedzać wartości najmniejsze, można postąpić dużo sprytniej. W poleceniu mamy podaną wartość $ x $, z którą mamy "porównać" k. element tablicy. Najlepszym rozwiązaniem jest odwiedzanie rekurencyjne wszystkich tych węzłów, których wartość jest mniejsza od $ x $. Zauważmy, że dużo łatwiej utworzyć funkcję rekurencyjną, która zwróci prawdę, jeżeli odwiedzony przez nas $ k. $ w kolejności węzeł będzie zawierał wartość mniejszą od $ x $, czyli nie będzie spełniał pożądanego warunku. Idea jest taka, że w rekurencyjnej funkcji dla bieżąvego węzła decydujemy, czy jest to węzeł o wartości mniejszej niż $ x $. Jeżeli nie, oznacza to, że dalsze odwiedzanie tego poddrzewa nie ma sensu, bo wszystkie kolejne wartości będą większe lub równe wartości $ x $, a my musimy jeszcze sprawdzić pozostałe węzły, czy wśród nich nie ma wartości mniejszych. Zatem zwracamy wówczas Fałsz, aby dalej nie przeglądać tego poddrzewa. Jeżeli jednak znajdujemy się w odpowiednim węźle, czyli takim, którego wartośc jest mniejsza od $ x $, zwiększamy licznik odwiedzonych węzłów o wartościach mniejszych od $ x $ o 1 i sprawdzamy, czy łączna liczba odwiedzonych przez nas węzłów wynosi $ k $, gdzie $ k $ oznacza, którą najmniejszą z kolei wartość szukamy (patrz polecenie). Jeżeli nasz licznik osiągnął wartość $ k $, oznacza to, że przejrzeliśmy dokładnie $ k $ węzłów o wartościach mniejszych niż $ x $, więc na pewno $ k. $ najmniejsza wartość nie może być większa lub równa $ x $, bo już było $ k $ wartości mniejszych. Zatem wtedy zwracamy prawdę, aby przerwać wywołania rekurencyjne. Jeżeli jednak wciąż wartość licznika jest mniejsza niż $ k $, sprawdzamy, czy bieżący węzeł ma prawe lub lewe dziecko i, jeżeli tak, przeszukujemy odpowiednie poddrzewa w sposób rekurencyjny.

Nie wiem, czy to, że dostajemy tablicę, która reprezentuje kopiec, oznacza, że jej wartości są odpowiednio ułożone, jak powinny być w kopcu, więc na początku buduję kopiec.

### Implementacja algorytmu

In [4]:
def is_kth_val_gt_or_eq_x(arr: list, k: int, x):
    # Return None if got not valid value of k
    if k < 1: return None
    heap = MinHeap(arr)
    
    # Traverse a heap till there are nodes of a value lower
    # than x or the number of nodes which we already visited
    # is lower than k
    visited_count = 0
    
    def traverse(idx):
        nonlocal visited_count
        
        if heap[idx] >= x:
            return False
        
        visited_count += 1
        if visited_count == k:
            return True

        l_idx = heap.left_child_idx(idx)
        r_idx = heap.right_child_idx(idx)

        if r_idx < heap.heap_size:
            return traverse(l_idx) or traverse(r_idx)
        if l_idx < heap.heap_size:
            return traverse(l_idx)
        
    return not traverse(0)
    

class MinHeap:
    def __init__(self, values):
        self.heap = values
        self.size = len(values)
        self._free_idx = self.size
        self.build_heap()
    
    def __bool__(self):
        return bool(self._free_idx)
    
    def __len__(self):
        return len(self.heap)
    
    def __getitem__(self, idx: int):
        if not isinstance(idx, int):
            raise TypeError(f"expected 'int', got {str(type(idx))[7:-1]}")
        if not 0 <= idx < self.heap_size:
            raise IndexError(f'node index too {"small" if idx < 0 else "large"}')
        return self.heap[idx]
    
    @property
    def heap_size(self):
        return self._free_idx
    
    @staticmethod
    def parent_idx(curr_idx):
        return (curr_idx - 1) // 2
    
    @staticmethod
    def left_child_idx(curr_idx):
        return curr_idx * 2 + 1
    
    @staticmethod
    def right_child_idx(curr_idx):
        return curr_idx * 2 + 2
    
    def swap(self, i, j):
        self.heap[i], self.heap[j] = self.heap[j], self.heap[i]
        
    def _heapify_down(self, curr_idx, end_idx):  # O(log n)
        # Loop till the current node has a child smaller than itself
        # We assume that when we enter a node which both children are
        # larger than this node, a subtree which a current node is a
        # root of must fulfill a min-heap property
        while True:
            l = self.left_child_idx(curr_idx)
            r = self.right_child_idx(curr_idx)
            smallest_idx = curr_idx

            if l < end_idx:
                if self.heap[l] < self.heap[curr_idx]: 
                    smallest_idx = l
                if r < end_idx and self.heap[r] < self.heap[smallest_idx]:
                    smallest_idx = r

            if smallest_idx != curr_idx:
                self.swap(curr_idx, smallest_idx)
                curr_idx = smallest_idx
            else:
                break
                
    def build_heap(self):   # O (n)
        for i in range(self.heap_size // 2 - 1, -1, -1):
            self._heapify_down(i, self.heap_size)

###### Kilka testów

In [5]:
import random

# random.seed(3)

arr = [random.randint(-100, 100) for _ in range(random.randint(1, 20))]
k = random.randrange(1, len(arr) + 1)
x = random.randint(-100, 100)

# arr = [-94, -74, -67, -33, -24, -21, -4, 45, 52, 75, 88, 97]
# x = -44
# k = 9

sorted_arr = sorted(arr)
print("Sorted arr:", sorted_arr, sep='\n')
print('x value:', x)
print('k value:', k)
print("Expected:", sorted_arr[k-1] >= x)
print("Result:  ", is_kth_val_gt_or_eq_x(arr, k, x))
print("Arr after calling function:", arr, sep='\n')
print("Has an array the same elements as before?:", not set(arr).symmetric_difference(sorted_arr))

Sorted arr:
[-72, -68, -57, -25, -24, -16, 6, 21, 23, 24, 29, 69, 74, 84, 95]
x value: 90
k value: 8
Expected: False
Result:   False
Arr after calling function:
[-72, -68, -57, 24, -16, -25, 21, 29, 74, 95, 6, -24, 84, 69, 23]
Has an array the same elements as before?: True


In [6]:
for _ in range(100_000):
    arr = [random.randint(-100, 100) for _ in range(random.randint(1, 20))]
    k = random.randrange(1, len(arr) + 1)
    x = random.randint(-100, 100)
    
    sorted_arr = sorted(arr)
    
    if (sorted_arr[k-1] >= x) is not is_kth_val_gt_or_eq_x(arr, k, x):
        print('Failed for:')
        print('k:', k)
        print('x:', x)
        print(arr)
        break
else:
    print("OK!")

OK!
